## Импорт библиотек

In [42]:
import re
import numpy as np
import sympy as sp
from abc import ABC, abstractmethod
from dataclasses import dataclass

## Интерфейс для задач оптимизации

In [43]:
class NLPProblemInterface(ABC):
    @property
    @abstractmethod
    def n(self):
        pass
    
    @property
    @abstractmethod
    def m(self):
        pass
    
    @property
    @abstractmethod
    def cl(self):
        pass
    
    @property
    @abstractmethod
    def cu(self):
        pass
    
    @property
    @abstractmethod
    def x0(self):
        pass
    
    @abstractmethod
    def obj(self, x):
        pass
    
    @abstractmethod
    def grad(self, x):
        pass
    
    @abstractmethod
    def cons(self, x):
        pass
    
    @abstractmethod
    def jac(self, x):
        pass
    
    @abstractmethod
    def hess(self, x, y):
        pass

## Реализация интерфейса для pycutest задач

In [44]:
class PyCUTestProblem(NLPProblemInterface):
    def __init__(self, cutest_problem, mu0=None, epsilon=None):
        self._prob = cutest_problem
        self._n = cutest_problem.n
        self._m = cutest_problem.m
        self._x0 = cutest_problem.x0.copy()
        
        if self._m > 0:
            self._cl = cutest_problem.cl.copy()
            self._cu = cutest_problem.cu.copy()
        else:
            self._cl = np.array([])
            self._cu = np.array([])
        
        self.mu0 = mu0
        self.epsilon = epsilon
    
    @property
    def n(self):
        return self._n
    
    @property
    def m(self):
        return self._m
    
    @property
    def cl(self):
        return self._cl
    
    @property
    def cu(self):
        return self._cu
    
    @property
    def x0(self):
        return self._x0
    
    def obj(self, x):
        return float(self._prob.obj(x))
    
    def grad(self, x):
        _, g = self._prob.obj(x, gradient=True)
        return np.asarray(g, dtype=float)
    
    def cons(self, x):
        if self._m == 0:
            return np.zeros(0, dtype=float)
        c = self._prob.cons(x)
        return np.asarray(c, dtype=float)
    
    def jac(self, x):
        if self._m == 0:
            return np.zeros((0, self._n), dtype=float)
        _, J = self._prob.cons(x, gradient=True)
        return np.asarray(J, dtype=float)
    
    def hess(self, x, y):
        if y.size != self._m:
            raise ValueError(f"hess: y has size {y.size} but m={self._m}")
        H = self._prob.hess(x, y)
        return np.asarray(H, dtype=float)

## Реализация интерфейса для txt-файлов

In [45]:
class TxtNLPProblem(NLPProblemInterface):
    def __init__(self, var_names, f_expr, c_exprs, cl, cu, x0,
                 s0=None, lam0=None, nu0=None, mu0=None, epsilon=None):
        self.var_names = list(var_names)
        self._n = len(self.var_names)

        self._x_syms = sp.symbols(self.var_names, real=True)
        x_syms = self._x_syms

        self._f_expr = f_expr
        self._f_func = sp.lambdify(x_syms, f_expr, "numpy")
        grad_expr = [sp.diff(f_expr, xi) for xi in x_syms]
        self._grad_func = sp.lambdify(x_syms, grad_expr, "numpy")
        self._Hf_expr = sp.hessian(f_expr, x_syms)

        self._c_exprs = list(c_exprs)
        self._m = len(self._c_exprs)
        self._cl = np.asarray(cl, dtype=float)
        self._cu = np.asarray(cu, dtype=float)

        self._c_func = sp.lambdify(x_syms, self._c_exprs, "numpy")
        J_expr = sp.Matrix(self._c_exprs).jacobian(x_syms)
        self._J_func = sp.lambdify(x_syms, J_expr, "numpy")

        self._Hc_exprs = [sp.hessian(ci, x_syms) for ci in self._c_exprs]

        self._x0 = np.asarray(x0, dtype=float)

        self.s0 = None if s0 is None else np.asarray(s0, dtype=float)
        self.lam0 = None if lam0 is None else np.asarray(lam0, dtype=float)
        self.nu0 = None if nu0 is None else np.asarray(nu0, dtype=float)
        self.mu0 = None if mu0 is None else float(mu0)
        self.epsilon = None if epsilon is None else float(epsilon)

    @property
    def n(self):
        return self._n
    
    @property
    def m(self):
        return self._m
    
    @property
    def cl(self):
        return self._cl
    
    @property
    def cu(self):
        return self._cu
    
    @property
    def x0(self):
        return self._x0

    @staticmethod
    def _parse_keyvals(line: str):
        key, val = line.split(":", 1)
        return key.strip(), val.strip()

    @staticmethod
    def _find_vars_in_exprs(expr_strs):
        names = set()
        for s in expr_strs:
            for m in re.findall(r"\bx\d+\b", s):
                names.add(m)

        def keyfn(v):
            return int(v[1:])
        return sorted(names, key=keyfn)

    @staticmethod
    def _make_sympy_locals(var_names):
        loc = {vn: sp.Symbol(vn, real=True) for vn in var_names}
        return loc

    @staticmethod
    def _parse_constraint_line(line: str):
        line = line.strip()
        if "<=" in line:
            left, right = line.split("<=", 1)
            return left.strip(), right.strip(), "<="
        if ">=" in line:
            left, right = line.split(">=", 1)
            return left.strip(), right.strip(), ">="
        if "=" in line:
            left, right = line.split("=", 1)
            return left.strip(), right.strip(), "="
        raise ValueError(f"Unsupported constraint format: {line}")

    @classmethod
    def from_file(cls, path: str):
        with open(path, "r", encoding="utf-8") as f:
            raw_lines = [ln.rstrip("\n") for ln in f]

        constraints_started = False
        header = []
        cons_lines = []
        for ln in raw_lines:
            if not ln.strip():
                continue
            if ln.strip().startswith("#"):
                continue
            if ln.strip().lower() == "constraints:":
                constraints_started = True
                continue
            if constraints_started:
                cons_lines.append(ln.strip())
            else:
                header.append(ln.strip())

        fields = {}
        for ln in header:
            if ":" in ln:
                k, v = cls._parse_keyvals(ln)
                fields[k.lower()] = v

        if "target_function" not in fields:
            raise ValueError("Missing 'target_function:'")
        f_str = fields["target_function"]

        expr_strs = [f_str]
        for c in cons_lines:
            left, right, op = cls._parse_constraint_line(c)
            expr_strs.append(left)
            expr_strs.append(right)

        var_names = cls._find_vars_in_exprs(expr_strs)
        if not var_names:
            raise ValueError("No variables like x1, x2, ... found.")

        loc = cls._make_sympy_locals(var_names)

        f_expr = sp.sympify(f_str, locals=loc)

        c_exprs = []
        cl = []
        cu = []
        for c in cons_lines:
            left, right, op = cls._parse_constraint_line(c)
            left_expr = sp.sympify(left, locals=loc)
            right_expr = sp.sympify(right, locals=loc)
            expr = left_expr - right_expr

            if op == "=":
                c_exprs.append(expr)
                cl.append(0.0)
                cu.append(0.0)
            elif op == "<=":
                c_exprs.append(expr)
                cl.append(-np.inf)
                cu.append(0.0)
            elif op == ">=":
                c_exprs.append(expr)
                cl.append(0.0)
                cu.append(np.inf)

        if "x" not in fields:
            raise ValueError("Missing 'x:' initial point.")
        x0 = [float(t) for t in fields["x"].split()]
        if len(x0) != len(var_names):
            raise ValueError(f"x has {len(x0)} values but detected {len(var_names)} variables: {var_names}")

        s0 = None
        if "s" in fields:
            s0 = [float(t) for t in fields["s"].split()]

        lam0 = None
        if "lambdas" in fields:
            lam0 = [float(t) for t in fields["lambdas"].split()]

        mu0 = None
        if "mu" in fields:
            mu0 = float(fields["mu"])

        eps = None
        if "epsilon" in fields:
            eps = float(fields["epsilon"])

        nu0 = None
        if "nu" in fields:
            nu0 = [float(t) for t in fields["nu"].split()]

        return cls(
            var_names=var_names,
            f_expr=f_expr,
            c_exprs=c_exprs,
            cl=cl,
            cu=cu,
            x0=x0,
            s0=s0,
            lam0=lam0,
            nu0=nu0,
            mu0=mu0,
            epsilon=eps,
        )

    def obj(self, x):
        x = np.asarray(x, dtype=float)
        return float(self._f_func(*x))

    def grad(self, x):
        x = np.asarray(x, dtype=float)
        g = self._grad_func(*x)
        return np.asarray(g, dtype=float).reshape(-1)

    def cons(self, x):
        x = np.asarray(x, dtype=float)
        if self._m == 0:
            return np.zeros(0, dtype=float)
        c = self._c_func(*x)
        return np.asarray(c, dtype=float).reshape(-1)

    def jac(self, x):
        x = np.asarray(x, dtype=float)
        if self._m == 0:
            return np.zeros((0, self._n), dtype=float)
        J = self._J_func(*x)
        return np.asarray(J, dtype=float)

    def hess(self, x, y):
        x = np.asarray(x, dtype=float)
        y = np.asarray(y, dtype=float)
        if y.size != self._m:
            raise ValueError(f"hess: y has size {y.size} but m={self._m}")

        H = np.asarray(sp.lambdify(self._x_syms, self._Hf_expr, "numpy")(*x), dtype=float)

        if self._m > 0:
            for i in range(self._m):
                if y[i] == 0.0:
                    continue
                Hi = np.asarray(sp.lambdify(self._x_syms, self._Hc_exprs[i], "numpy")(*x), dtype=float)
                H = H + y[i] * Hi
        return H

## Вспомогательный класс для разделения ограничений (используется внутри решателя)

In [46]:
class _ConstraintMapper:
    def __init__(self, prob, tol_eq=0.0):
        self.prob = prob
        self.tol_eq = tol_eq

        m = prob.m
        cl = np.asarray(prob.cl, dtype=float).copy()
        cu = np.asarray(prob.cu, dtype=float).copy()

        self.eq_idx = []
        self.ineq_upper = []
        self.ineq_lower = []
        self.free_idx = []

        for i in range(m):
            li, ui = cl[i], cu[i]
            li_f = np.isfinite(li)
            ui_f = np.isfinite(ui)

            if li_f and ui_f and abs(li - ui) <= tol_eq:
                self.eq_idx.append(i)
            else:
                has_ineq = False
                if ui_f:
                    self.ineq_upper.append((i, ui))
                    has_ineq = True
                if li_f:
                    self.ineq_lower.append((i, li))
                    has_ineq = True
                if not has_ineq:
                    self.free_idx.append(i)

        self.m_eq = len(self.eq_idx)
        self.p_ineq = len(self.ineq_upper) + len(self.ineq_lower)

        self.g_spec = []
        for (i, u) in self.ineq_upper:
            self.g_spec.append((i, +1, u))
        for (i, l) in self.ineq_lower:
            self.g_spec.append((i, -1, l))

    def split_constraints(self, x):
        c = np.asarray(self.prob.cons(x), dtype=float)
        h = np.zeros(self.m_eq, dtype=float)
        for k, i in enumerate(self.eq_idx):
            h[k] = c[i] - self.prob.cl[i]

        g = np.zeros(self.p_ineq, dtype=float)
        for j, (i, kind, bnd) in enumerate(self.g_spec):
            if kind == +1:
                g[j] = c[i] - bnd
            else:
                g[j] = bnd - c[i]
        return h, g

    def split_jacobians(self, x):
        J = np.asarray(self.prob.jac(x), dtype=float)
        n = self.prob.n

        Jh = np.zeros((self.m_eq, n), dtype=float)
        for k, i in enumerate(self.eq_idx):
            Jh[k, :] = J[i, :]

        Jg = np.zeros((self.p_ineq, n), dtype=float)
        for j, (i, kind, _) in enumerate(self.g_spec):
            if kind == +1:
                Jg[j, :] = J[i, :]
            else:
                Jg[j, :] = -J[i, :]
        return Jh, Jg

    def multipliers_to_original(self, lam, nu):
        y = np.zeros(self.prob.m, dtype=float)

        for k, i in enumerate(self.eq_idx):
            y[i] += lam[k]

        for j, (i, kind, _) in enumerate(self.g_spec):
            if kind == +1:
                y[i] += nu[j]
            else:
                y[i] += -nu[j]
        return y

In [47]:
@dataclass(frozen=True)
class Solution:
    x: np.ndarray
    s: np.ndarray
    lam: np.ndarray
    nu: np.ndarray
    mu: float
    f: float

## Метод внутренней точки

In [48]:
class InteriorPointMethod:
    def __init__(
        self,
        prob,
        mu0=1e-1,
        sigma=0.1,
        newton_tol=1e-8,
        outer_tol=1e-8,
        mu_min=1e-6,
        max_newton_iters=1000,
        max_outer_iters=50,
        alpha_backtrack=0.5,
        alpha_min=1e-10,
        positivity_eta=0.99,
        verbose=True,
    ):
        if not isinstance(prob, NLPProblemInterface):
            raise TypeError("prob должен реализовывать NLPProblemInterface")
        
        self.prob = prob
        self._mapper = _ConstraintMapper(prob)

        if getattr(prob, "mu0", None) is not None:
            mu0 = prob.mu0
        if getattr(prob, "epsilon", None) is not None:
            outer_tol = prob.epsilon

        self.mu0 = float(mu0)
        self.sigma = float(sigma)
        self.newton_tol = float(newton_tol)
        self.outer_tol = float(outer_tol)
        self.mu_min = float(mu_min)
        self.max_newton_iters = int(max_newton_iters)
        self.max_outer_iters = int(max_outer_iters)
        self.alpha_backtrack = float(alpha_backtrack)
        self.alpha_min = float(alpha_min)
        self.positivity_eta = float(positivity_eta)
        self.verbose = bool(verbose)

    def _F_and_J(self, x, s, lam, nu, mu):
        n = self.prob.n
        m = self._mapper.m_eq
        p = self._mapper.p_ineq

        grad_f = np.asarray(self.prob.grad(x), dtype=float)
        h, g = self._mapper.split_constraints(x)
        Jh, Jg = self._mapper.split_jacobians(x)

        r_dual = grad_f.copy()
        if m > 0:
            r_dual += Jh.T @ lam
        if p > 0:
            r_dual += Jg.T @ nu

        r_cent = s * nu - mu * np.ones(p, dtype=float)
        r_pri_eq = h.copy()
        r_pri_ineq = g + s

        F = np.concatenate([r_dual, r_cent, r_pri_eq, r_pri_ineq])

        y = self._mapper.multipliers_to_original(lam, nu)
        H = np.asarray(self.prob.hess(x, y), dtype=float)

        dim = n + p + m + p
        J = np.zeros((dim, dim), dtype=float)

        ix0, ix1 = 0, n
        is0, is1 = ix1, ix1 + p
        il0, il1 = is1, is1 + m
        inu0, inu1 = il1, il1 + p

        J[ix0:ix1, ix0:ix1] = H
        if m > 0:
            J[ix0:ix1, il0:il1] = Jh.T
        if p > 0:
            J[ix0:ix1, inu0:inu1] = Jg.T

        if p > 0:
            J[is0:is1, is0:is1] = np.diag(nu)
            J[is0:is1, inu0:inu1] = np.diag(s)

        if m > 0:
            J[il0:il1, ix0:ix1] = Jh

        if p > 0:
            J[inu0:inu1, ix0:ix1] = Jg
            J[inu0:inu1, is0:is1] = np.eye(p)

        return F, J

    def _initial_point(self, mu):
        n = self.prob.n
        x = np.asarray(self.prob.x0, dtype=float).copy()

        _, g = self._mapper.split_constraints(x)
        p = self._mapper.p_ineq

        s0 = getattr(self.prob, "s0", None)
        lam0 = getattr(self.prob, "lam0", None)
        nu0 = getattr(self.prob, "nu0", None)

        if p > 0:
            if s0 is not None and len(s0) == p:
                s = np.asarray(s0, dtype=float).copy()
                s = np.maximum(s, 1e-8)
            else:
                s = np.maximum(1.0, -g + 1.0)

            if nu0 is not None and len(nu0) == p:
                nu = np.asarray(nu0, dtype=float).copy()
                nu = np.maximum(nu, 1e-8)
            else:
                nu = mu / s
        else:
            s = np.zeros(0, dtype=float)
            nu = np.zeros(0, dtype=float)

        m = self._mapper.m_eq
        if m > 0:
            if lam0 is not None and len(lam0) == m:
                lam = np.asarray(lam0, dtype=float).copy()
            else:
                lam = np.zeros(m, dtype=float)
        else:
            lam = np.zeros(0, dtype=float)

        return x, s, lam, nu

    def solve(self):
        mu = self.mu0
        x, s, lam, nu = self._initial_point(mu)

        n = self.prob.n
        m = self._mapper.m_eq
        p = self._mapper.p_ineq

        def normF(x_, s_, lam_, nu_, mu_):
            F_, _ = self._F_and_J(x_, s_, lam_, nu_, mu_)
            return np.linalg.norm(F_, ord=2)

        for outer_it in range(1, self.max_outer_iters + 1):
            for newton_it in range(1, self.max_newton_iters + 1):
                F, J = self._F_and_J(x, s, lam, nu, mu)
                F_norm = np.linalg.norm(F, ord=2)

                if self.verbose:
                    print(f"[outer {outer_it:02d}] mu={mu:.3e}  "
                          f"[newton {newton_it:02d}] ||F||={F_norm:.3e}")

                if F_norm <= self.newton_tol:
                    break

                try:
                    dz = np.linalg.solve(J, -F)
                except np.linalg.LinAlgError:
                    dz, *_ = np.linalg.lstsq(J, -F, rcond=None)

                dx = dz[0:n]
                ds = dz[n:n+p]
                dlam = dz[n+p:n+p+m]
                dnu = dz[n+p+m:n+p+m+p]

                alpha = 1.0

                if p > 0:
                    neg_ds = ds < 0
                    neg_dnu = dnu < 0
                    if np.any(neg_ds):
                        alpha = min(alpha, self.positivity_eta * np.min(-s[neg_ds] / ds[neg_ds]))
                    if np.any(neg_dnu):
                        alpha = min(alpha, self.positivity_eta * np.min(-nu[neg_dnu] / dnu[neg_dnu]))
                    alpha = min(alpha, 1.0)

                base_norm = F_norm
                while alpha >= self.alpha_min:
                    x_new = x + alpha * dx
                    s_new = s + alpha * ds
                    lam_new = lam + alpha * dlam
                    nu_new = nu + alpha * dnu

                    if p > 0 and (np.any(s_new <= 0.0) or np.any(nu_new <= 0.0)):
                        alpha *= self.alpha_backtrack
                        continue

                    new_norm = normF(x_new, s_new, lam_new, nu_new, mu)
                    if new_norm > base_norm:
                        alpha *= self.alpha_backtrack
                        continue

                    x, s, lam, nu = x_new, s_new, lam_new, nu_new
                    break

                if alpha < self.alpha_min:
                    if self.verbose:
                        print("Step size underflow; stopping Newton for this mu.")
                    break

            F_norm = normF(x, s, lam, nu, mu)
            if (mu <= self.mu_min) and (F_norm <= self.outer_tol):
                if self.verbose:
                    print("Converged.")
                break

            mu = self.sigma * mu
            if p > 0:
                nu = np.maximum(nu, 1e-16)
                s = np.maximum(s, 1e-16)

        f_star = float(self.prob.obj(x))

        return Solution(
            x=x,
            s=s,
            lam=lam,
            nu=nu,
            mu=mu,
            f=f_star
        )

## Примеры использования

### Пример 1: Решение задачи из txt-файла

In [49]:
txt_problem = TxtNLPProblem.from_file("input.txt")
solver = InteriorPointMethod(txt_problem, verbose=True)
solution = solver.solve()

print("\nРезультаты для задачи из txt:")
print("x* =", solution.x)
print("f(x*) =", solution.f)

[outer 01] mu=1.000e-01  [newton 01] ||F||=3.754e+00
[outer 01] mu=1.000e-01  [newton 02] ||F||=1.748e+00
[outer 01] mu=1.000e-01  [newton 03] ||F||=3.833e-01
[outer 01] mu=1.000e-01  [newton 04] ||F||=4.136e-02
[outer 01] mu=1.000e-01  [newton 05] ||F||=2.616e-03
[outer 01] mu=1.000e-01  [newton 06] ||F||=1.521e-05
[outer 01] mu=1.000e-01  [newton 07] ||F||=5.203e-10
[outer 02] mu=1.000e-02  [newton 01] ||F||=1.273e-01
[outer 02] mu=1.000e-02  [newton 02] ||F||=2.348e-02
[outer 02] mu=1.000e-02  [newton 03] ||F||=3.853e-03
[outer 02] mu=1.000e-02  [newton 04] ||F||=2.379e-04
[outer 02] mu=1.000e-02  [newton 05] ||F||=1.241e-06
[outer 02] mu=1.000e-02  [newton 06] ||F||=3.448e-11
[outer 03] mu=1.000e-03  [newton 01] ||F||=1.273e-02
[outer 03] mu=1.000e-03  [newton 02] ||F||=2.272e-03
[outer 03] mu=1.000e-03  [newton 03] ||F||=3.796e-04
[outer 03] mu=1.000e-03  [newton 04] ||F||=2.397e-05
[outer 03] mu=1.000e-03  [newton 05] ||F||=1.258e-07
[outer 03] mu=1.000e-03  [newton 06] ||F||=3.5

### Пример 2: Решение задачи из pycutest

In [ ]:
!pip install meson
!export PATH="$HOME/.local/bin:$PATH"
!meson --version

In [ ]:
!sudo apt -y update
!sudo apt -y install ninja-build
!ninja --version

In [ ]:
!mkdir cutest
%cd cutest
!git clone https://github.com/ralna/SIFDecode ./sifdecode
!git clone https://github.com/ralna/CUTEst ./cutest
!git clone https://bitbucket.org/optrove/sif ./mastsif

In [ ]:
%cd sifdecode
!meson setup builddir
!meson compile -C builddir
!sudo meson install -C builddir

In [ ]:
%cd ../cutest
!meson setup builddir -Dmodules=false
!meson compile -C builddir
!sudo meson install -C builddir

In [ ]:
%env MASTSIF=/content/cutest/mastsif

In [ ]:
!pip install pycutest

In [ ]:
import pycutest

In [ ]:
cutest_problem_name = "HS21"

cutest_raw = pycutest.import_problem(cutest_problem_name)
cutest_problem = PyCUTestProblem(cutest_raw)

solver = InteriorPointMethod(cutest_problem, mu0=1e-1, sigma=0.1, verbose=True)
solution = solver.solve()

print(f"Результаты для задачи {cutest_problem_name} из pycutest:")
print("x* =", solution.x)
print("f(x*) =", solution.f)